<a href="https://colab.research.google.com/github/sinan-894/git_test/blob/main/better_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/jordan-bird/CIFAKE-Real-and-AI-Generated-Synthetic-Images.git
%ls

Cloning into 'CIFAKE-Real-and-AI-Generated-Synthetic-Images'...
remote: Enumerating objects: 118694, done.
remote: Counting objects: 100% (118694/118694), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 118694 (delta 118651), reused 118673 (delta 118645), pack-reused 0 (from 0)
Receiving objects: 100% (118694/118694), 39.68 MiB | 11.03 MiB/s, done.
Resolving deltas: 100% (118651/118651), done.
Updating files: 100% (120001/120001), done.
CIFAKE-Real-and-AI-Generated-Synthetic-Images/  sample_data/


In [2]:
import numpy as np
import tensorflow as tf

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2
from sklearn.preprocessing import LabelEncoder , OneHotEncoder


PATH_CURRENT = '/content/CIFAKE-Real-and-AI-Generated-Synthetic-Images/DATASET/train'



dataset_path = os.listdir(PATH_CURRENT)

print (dataset_path)  #what kinds of classes are in this dataset

print("Types of classes labels found: ", len(dataset_path))

class_labels = []

for item in dataset_path:
 # Get all the file names
    all_classes = os.listdir(PATH_CURRENT + '/' +item)
 #print(all_classes)

 # Add them to the list
    for room in all_classes:
        class_labels.append((item, str('dataset_path' + '/' +item) + '/' + room))

df = pd.DataFrame(data=class_labels, columns=['Labels', 'image'])
print("Total number of images in the dataset: ", len(df))

label_count = df['Labels'].value_counts()
print(label_count)

path = PATH_CURRENT + '/'
dataset_path = os.listdir(PATH_CURRENT)

im_size = 32

images = []
labels = []

for i in dataset_path:
    data_path = path + str(i)
    filenames = [i for i in os.listdir(data_path) ]

    for f in filenames:
        img = cv2.imread(data_path + '/' + f)
        img = cv2.resize(img, (im_size, im_size))
        images.append(img)
        labels.append(i)

images = np.array(images)

images = images.astype('float32') / 255.0



y=df['Labels'].values
print(y)

y_labelencoder = LabelEncoder ()
y = y_labelencoder.fit_transform (y)

y=y.reshape(-1,1)

['REAL', 'FAKE']
Types of classes labels found:  2
Total number of images in the dataset:  100000
Labels
REAL    50000
FAKE    50000
Name: count, dtype: int64
['REAL' 'REAL' 'REAL' ... 'FAKE' 'FAKE' 'FAKE']


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Model Creation

In [6]:
import tensorflow as tf
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam

# Define the CNN model
def create_cnn_model(input_shape=(32, 32, 3)):
    inputs = Input(shape=input_shape)
    x = Conv2D(32, (3,3), activation='relu', padding='same')(inputs)
    x = Conv2D(64, (3,3), activation='relu', padding='same')(x)
    x = MaxPooling2D(pool_size=(2,2))(x)

    x = Conv2D(64, (3,3), activation='relu', padding='same')(x)
    x = Conv2D(64, (3,3), activation='relu', padding='same')(x)
    x = MaxPooling2D(pool_size=(2,2))(x)

    x = Conv2D(128, (3,3), activation='relu', padding='same')(x)
    x = Conv2D(128, (3,3), activation='relu', padding='same')(x)
    x = MaxPooling2D(pool_size=(2,2))(x)

    x = Flatten()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(16, activation='relu')(x)
    outputs = Dense(1, activation='sigmoid')(x)  # Binary classification (Real vs AI-generated)

    model = Model(inputs, outputs)
    model.compile(optimizer=Adam(learning_rate=0.0001),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    return model


cnn_model = create_cnn_model()

# Print model summary
cnn_model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 32, 32, 3)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d (Conv2D)                      │ (None, 32, 32, 32)          │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 32, 32, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 16, 16, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 16, 16, 64)          │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 16, 16, 64)          │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 8, 8, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 8, 8, 128)           │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 8, 8, 128)           │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 4, 4, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 2048)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │       1,049,088 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 16)                  │           4,112 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │              17 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,499,233 (5.72 MB)

 Trainable params: 1,499,233 (5.72 MB)

 Non-trainable params: 0 (0.00 B)

In [7]:
hist_train=  cnn_model.fit(images, y, epochs=30, verbose=2)

Epoch 1/30
3125/3125 - 24s - 8ms/step - accuracy: 0.8606 - loss: 0.3164
Epoch 2/30
3125/3125 - 36s - 12ms/step - accuracy: 0.9262 - loss: 0.1906
Epoch 3/30
3125/3125 - 21s - 7ms/step - accuracy: 0.9393 - loss: 0.1599
Epoch 4/30
3125/3125 - 20s - 7ms/step - accuracy: 0.9469 - loss: 0.1392
Epoch 5/30
3125/3125 - 21s - 7ms/step - accuracy: 0.9532 - loss: 0.1246
Epoch 6/30
3125/3125 - 20s - 6ms/step - accuracy: 0.9573 - loss: 0.1139
Epoch 7/30
3125/3125 - 17s - 5ms/step - accuracy: 0.9618 - loss: 0.1024
Epoch 8/30
3125/3125 - 17s - 5ms/step - accuracy: 0.9651 - loss: 0.0919
Epoch 9/30
3125/3125 - 17s - 5ms/step - accuracy: 0.9694 - loss: 0.0823
Epoch 10/30
3125/3125 - 17s - 5ms/step - accuracy: 0.9733 - loss: 0.0730
Epoch 11/30
3125/3125 - 21s - 7ms/step - accuracy: 0.9765 - loss: 0.0635
Epoch 12/30
3125/3125 - 17s - 5ms/step - accuracy: 0.9794 - loss: 0.0546
Epoch 13/30
3125/3125 - 17s - 5ms/step - accuracy: 0.9819 - loss: 0.0490
Epoch 14/30
3125/3125 - 20s - 7ms/step - accuracy: 0.9844 -

In [8]:
cnn_model.save('cnn1.keras')

In [9]:
import matplotlib.pyplot as plt

|
def plot_hist(hist):
    plt.plot(hist.history["accuracy"])
    #plt.plot(hist.history["val_accuracy"])
    plt.title("model accuracy")
    plt.ylabel("accuracy")
    plt.xlabel("epoch")
    plt.legend(["train", "validation"], loc="upper left")
    plt.show()

plot_hist(hist_train)

SyntaxError: invalid syntax (<ipython-input-9-fb63a28fda60>, line 3)

In [10]:
PATH_CURRENT = PATH_CURRENT = '/content/CIFAKE-Real-and-AI-Generated-Synthetic-Images/DATASET/test'



dataset_path = os.listdir(PATH_CURRENT)

print (dataset_path)  #what kinds of classes are in this dataset

print("Types of classes labels found: ", len(dataset_path))

class_labels = []

for item in dataset_path:
 # Get all the file names
 all_classes = os.listdir(PATH_CURRENT + '/' +item)
 #print(all_classes)

 # Add them to the list
 for room in all_classes:
    class_labels.append((item, str('dataset_path' + '/' +item) + '/' + room))

df = pd.DataFrame(data=class_labels, columns=['Labels', 'image'])

print("Total number of images in the dataset: ", len(df))

label_count = df['Labels'].value_counts()
print(label_count)

import cv2
path = PATH_CURRENT + '/'
dataset_path = os.listdir(PATH_CURRENT)

im_size = 32

images = []
labels = []

for i in dataset_path:
    data_path = path + str(i)
    filenames = [i for i in os.listdir(data_path) ]

    for f in filenames:
        img = cv2.imread(data_path + '/' + f)
        img = cv2.resize(img, (im_size, im_size))
        images.append(img)
        labels.append(i)
images = np.array(images)
images = images.astype('float32') / 255.0
# print(images[0])
print(images.shape)

y=df['Labels'].values
print(y)

y_labelencoder = LabelEncoder ()
y = y_labelencoder.fit_transform (y)
print (y)
print(y.shape)

y=y.reshape(-1,1)

['REAL', 'FAKE']
Types of classes labels found:  2
Total number of images in the dataset:  20000
Labels
REAL    10000
FAKE    10000
Name: count, dtype: int64
(20000, 32, 32, 3)
['REAL' 'REAL' 'REAL' ... 'FAKE' 'FAKE' 'FAKE']
[1 1 1 ... 0 0 0]
(20000,)


In [11]:
preds = cnn_model.evaluate(images, y)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

625/625 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.9624 - loss: 0.1767
Loss = 0.1801992654800415
Test Accuracy = 0.9613500237464905
